|Your API key is: RA9208IQ9J5IF8AS

This API returns a list of active or delisted US stocks and ETFs, either as of the latest trading day or at a specific time in history. The endpoint is positioned to facilitate equity research on asset lifecycle and survivorship. 

In [17]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey= RA9208IQ9J5IF8AS'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

['symbol', 'name', 'exchange', 'assetType', 'ipoDate', 'delistingDate', 'status']
['A', 'Agilent Technologies Inc', 'NYSE', 'Stock', '1999-11-18', 'null', 'Active']
['AA', 'Alcoa Corp', 'NYSE', 'Stock', '2016-10-18', 'null', 'Active']
['AAA', 'AAF First Priority CLO Bond ETF', 'NYSE ARCA', 'ETF', '2020-09-09', 'null', 'Active']
['AAAU', 'Goldman Sachs Physical Gold ETF', 'NYSE ARCA', 'ETF', '2018-08-15', 'null', 'Active']
['AAC', 'Ares Acquisition Corporation - Class A', 'NYSE', 'Stock', '2021-03-25', 'null', 'Active']
['AAC-U', 'Ares Acquisition Corporation - Units (1 Ord Share Class A & 1/5 War)', 'NYSE', 'Stock', '2021-02-02', 'null', 'Active']
['AAC-WS', 'Ares Acquisition Corporation - Warrants (01/01/9999)', 'NYSE', 'Stock', '2021-03-25', 'null', 'Active']
['AACG', 'ATA Creativity Global', 'NASDAQ', 'Stock', '2008-01-29', 'null', 'Active']
['AACI', 'Armada Acquisition Corp I', 'NASDAQ', 'Stock', '2021-11-10', 'null', 'Active']
['AACIU', 'Armada Acquisition Corp I - Units (1 Ord & 

['BICK', 'First Trust BICK Index Fund', 'NASDAQ', 'ETF', '2010-04-13', 'null', 'Active']
['BIDS', 'Amplify Digital & Online Trading ETF', 'NYSE ARCA', 'ETF', '2021-09-22', 'null', 'Active']
['BIDU', 'Baidu Inc', 'NASDAQ', 'Stock', '2005-08-05', 'null', 'Active']
['BIF', 'Boulder Growth & Income Fund Inc', 'NYSE', 'ETF', '1988-01-05', 'null', 'Active']
['BIG', 'Big Lots Inc', 'NYSE', 'Stock', '1985-06-20', 'null', 'Active']
['BIGC', 'BigCommerce Holdings Inc Series 1', 'NASDAQ', 'Stock', '2020-08-05', 'null', 'Active']
['BIGY', 'Defiance Next Gen Big Data ETF', 'NYSE ARCA', 'ETF', '2021-07-14', 'null', 'Active']
['BIGZ', 'BlackRock Innovation & Growth Trust', 'NYSE', 'Stock', '2021-03-26', 'null', 'Active']
['BIIB', 'Biogen Inc', 'NASDAQ', 'Stock', '1991-09-17', 'null', 'Active']
['BIL', 'SPDR Bloomberg 1-3 Month T-Bill ETF', 'NYSE ARCA', 'ETF', '2007-05-30', 'null', 'Active']
['BILI', 'Bilibili Inc', 'NASDAQ', 'Stock', '2018-03-28', 'null', 'Active']
['BILL', 'Bill.com Holdings Inc', '

['CNOBP', 'ConnectOne Bancorp Inc', 'NASDAQ', 'Stock', '2021-08-13', 'null', 'Active']
['CNP', 'Centerpoint Energy Inc', 'NYSE', 'Stock', '1970-01-02', 'null', 'Active']
['CNQ', 'Canadian Natural Resources Ltd', 'NYSE', 'Stock', '2000-07-31', 'null', 'Active']
['CNR', 'Cornerstone Building Brands Inc', 'NYSE', 'Stock', '1992-04-06', 'null', 'Active']
['CNRG', 'SPDR S&P Kensho Clean Power ETF', 'NYSE ARCA', 'ETF', '2018-10-23', 'null', 'Active']
['CNS', 'Cohen & Steers Inc', 'NYSE', 'Stock', '2004-08-16', 'null', 'Active']
['CNSL', 'Consolidated Communications Holdings Inc', 'NASDAQ', 'Stock', '2005-07-22', 'null', 'Active']
['CNSP', 'Cns Pharmaceuticals Inc', 'NASDAQ', 'Stock', '2019-11-08', 'null', 'Active']
['CNTA', 'Centessa Pharmaceuticals plc', 'NASDAQ', 'Stock', '2021-05-28', 'null', 'Active']
['CNTB', 'Connect Biopharma Holdings Ltd', 'NASDAQ', 'Stock', '2021-03-19', 'null', 'Active']
['CNTG', 'Centogene NV', 'NASDAQ', 'Stock', '2019-11-07', 'null', 'Active']
['CNTQ', 'Chardan N

['EEV', 'ProShares UltraShort MSCI Emerging Markets', 'NYSE ARCA', 'ETF', '2007-11-01', 'null', 'Active']
['EEX', 'Emerald Holding Inc', 'NYSE', 'Stock', '2017-04-28', 'null', 'Active']
['EFA', 'iShares MSCI EAFE ETF', 'NYSE ARCA', 'ETF', '2001-08-17', 'null', 'Active']
['EFAD', 'ProShares MSCI EAFE Dividend Growers ETF', 'BATS', 'ETF', '2014-08-21', 'null', 'Active']
['EFAS', 'Global X MSCI SuperDividend EAFE ETF', 'NASDAQ', 'ETF', '2016-11-16', 'null', 'Active']
['EFAV', 'BTC iShares MSCI EAFE Min Vol Factor ETF', 'BATS', 'ETF', '2011-10-20', 'null', 'Active']
['EFAX', 'SPDR MSCI EAFE Fossil Fuel Reserves Free ETF', 'NYSE ARCA', 'ETF', '2016-10-25', 'null', 'Active']
['EFC', 'Ellington Financial Inc', 'NYSE', 'Stock', '2010-10-08', 'null', 'Active']
['EFC-P-A', 'Ellington Financial Inc', 'NYSE', 'Stock', '2019-10-17', 'null', 'Active']
['EFF', 'Eaton Vance FloatingRate Income Plus Fund', 'NYSE', 'ETF', '2013-06-26', 'null', 'Active']
['EFG', 'BTC iShares MSCI EAFE Growth ETF', 'BATS'

['FATP', 'Fat Projects Acquisition Corp - Class A', 'NASDAQ', 'Stock', '2021-12-03', 'null', 'Active']
['FATPU', 'Fat Projects Acquisition Corp - Units (1 Ord Share Class A & 1 War)', 'NASDAQ', 'Stock', '2021-10-13', 'null', 'Active']
['FATPW', 'Fat Projects Acquisition Corp - Warrants (31/08/2026)', 'NASDAQ', 'Stock', '2021-12-03', 'null', 'Active']
['FATT', 'Fat Tail Risk ETF', 'NYSE ARCA', 'ETF', '2021-05-25', 'null', 'Active']
['FAUG', 'FT Cboe Vest U.S. Equity Buffer ETF - August', 'BATS', 'ETF', '2019-11-07', 'null', 'Active']
['FAX', 'Aberdeen Asia-Pacific Income Fund Inc', 'NYSE MKT', 'ETF', '1988-02-10', 'null', 'Active']
['FAZ', 'Direxion Daily Financial Bear 3x Shares', 'NYSE ARCA', 'ETF', '2008-11-19', 'null', 'Active']
['FB', 'Meta Platforms Inc - Class A', 'NASDAQ', 'Stock', '2012-05-18', 'null', 'Active']
['FBC', 'Flagstar Bancorp Inc', 'NYSE', 'Stock', '1997-04-30', 'null', 'Active']
['FBCG', 'Fidelity Blue Chip Growth ETF', 'BATS', 'ETF', '2020-06-04', 'null', 'Active'

...

['PCG-P-G', 'Pacific Gas & Electric Company', 'NYSE MKT', 'Stock', '2014-09-22', 'null', 'Active']
['PCG-P-H', 'Pacific Gas & Electric Company', 'NYSE MKT', 'Stock', '2014-09-22', 'null', 'Active']
['PCG-P-I', 'Pacific Gas & Electric Company', 'NYSE MKT', 'Stock', '2014-09-22', 'null', 'Active']
['PCGU', 'PG&E Corp', 'NYSE', 'Stock', '2020-06-26', 'null', 'Active']
['PCH', 'PotlatchDeltic Corp', 'NASDAQ', 'Stock', '1985-11-11', 'null', 'Active']
['PCI', 'PIMCO Dynamic Credit and Mortgage Income Fund', 'NYSE', 'ETF', '2013-01-29', 'null', 'Active']
['PCK', 'Pimco California Municipal Income Fund II', 'NYSE', 'ETF', '2002-06-26', 'null', 'Active']
['PCM', 'PCM Fund Inc', 'NYSE', 'ETF', '1993-08-27', 'null', 'Active']
['PCN', 'PIMCO Corporate & Income Strategy Fund', 'NYSE', 'ETF', '2001-12-19', 'null', 'Active']
['PCOM', 'Points International Ltd', 'NASDAQ', 'Stock', '2011-02-10', 'null', 'Active']
['PCOR', 'Procore Technologies Inc', 'NYSE', 'Stock', '2021-05-20', 'null', 'Active']
['PC

['RMED', 'Ra Medical Systems Inc', 'NYSE MKT', 'Stock', '2018-09-27', 'null', 'Active']
['RMG-U', 'Romeo Power Inc - Unit (1 Ord Cls A & 0.33 Wrt)', 'NYSE', 'Stock', '2019-02-08', 'null', 'Active']
['RMGC', 'RMG Acquisition Corp III - Class A', 'NASDAQ', 'Stock', '2021-03-30', 'null', 'Active']
['RMGCU', 'RMG Acquisition Corp III - Units (1 Ord Share Class A & 1/5 War)', 'NASDAQ', 'Stock', '2021-02-05', 'null', 'Active']
['RMGCW', 'RMG Acquisition Corp III - Warrants (08/02/2026)', 'NASDAQ', 'Stock', '2021-03-30', 'null', 'Active']
['RMI', 'RiverNorth Opportunistic Municipal Income Fund Inc', 'NYSE', 'Stock', '2018-10-26', 'null', 'Active']
['RMM', 'RiverNorth Managed Duration Municipal Income Fund Inc', 'NYSE', 'Stock', '2019-07-26', 'null', 'Active']
['RMNI', 'Rimini Street Inc', 'NASDAQ', 'Stock', '2015-08-28', 'null', 'Active']
['RMO', 'Romeo Power Inc - Class A', 'NYSE', 'Stock', '2019-04-01', 'null', 'Active']
['RMO-WS', 'Romeo Power Inc Warrants', 'NYSE', 'Stock', '2020-12-30', 

['STRM', 'Streamline Health Solutions Inc', 'NASDAQ', 'Stock', '1996-04-18', 'null', 'Active']
['STRN', 'Stran & Company Inc', 'NASDAQ', 'Stock', '2021-11-09', 'null', 'Active']
['STRNW', 'Stran & Company Inc - Warrants (01/11/2026)', 'NASDAQ', 'Stock', '2021-11-09', 'null', 'Active']
['STRO', 'Sutro Biopharma Inc', 'NASDAQ', 'Stock', '2018-09-27', 'null', 'Active']
['STRR', 'Star Equity Holdings Inc', 'NASDAQ', 'Stock', '2004-06-10', 'null', 'Active']
['STRRP', 'Star Equity Holdings Inc', 'NASDAQ', 'Stock', '2019-09-13', 'null', 'Active']
['STRS', 'Stratus Properties Inc', 'NASDAQ', 'Stock', '1993-03-04', 'null', 'Active']
['STRT', 'Strattec Security Corp', 'NASDAQ', 'Stock', '1995-02-24', 'null', 'Active']
['STSA', 'Satsuma Pharmaceuticals Inc', 'NASDAQ', 'Stock', '2019-09-13', 'null', 'Active']
['STT', 'State Street Corp', 'NYSE', 'Stock', '1986-07-09', 'null', 'Active']
['STT-P-C', 'State Street Corporation Dep Shs Representing 14000 Ownership Int In Sh Non Cum Perpertual Pfd Stk S

['TRITW', 'Triterras Inc - Warrants (10/11/2025)', 'NASDAQ', 'Stock', '2020-11-11', 'null', 'Active']
['TRKA', 'Troika Media Group Inc', 'NASDAQ', 'Stock', '2021-04-20', 'null', 'Active']
['TRKAW', 'Troika Media Group Inc - Warrants (13/04/2024)', 'NASDAQ', 'Stock', '2021-04-20', 'null', 'Active']
['TRMB', 'Trimble Inc', 'NASDAQ', 'Stock', '1992-03-03', 'null', 'Active']
['TRMD', 'Torm Plc - Class A', 'NASDAQ', 'Stock', '2018-02-23', 'null', 'Active']
['TRMK', 'Trustmark Corp', 'NASDAQ', 'Stock', '1992-03-03', 'null', 'Active']
['TRMR', 'Tremor International Ltd', 'NASDAQ', 'Stock', '2021-06-18', 'null', 'Active']
['TRN', 'Trinity Industries Inc', 'NYSE', 'Stock', '1987-12-30', 'null', 'Active']
['TRND', 'Pacer Trendpilot Fund of Funds ETF', 'NYSE ARCA', 'ETF', '2019-05-06', 'null', 'Active']
['TRNE-U', 'Desktop Metal Inc - Unit (1 Ord share Cls A & 1/2 Wrt)', 'NYSE', 'Stock', '2019-03-15', 'null', 'Active']
['TRNO', 'Terreno Realty Corp', 'NYSE', 'Stock', '2010-02-10', 'null', 'Active

['VGZ', 'Vista Gold Corp', 'NYSE MKT', 'Stock', '1990-04-30', 'null', 'Active']
['VHAQ', 'Viveon Health Acquisition Corp', 'NYSE MKT', 'Stock', '2021-02-04', 'null', 'Active']
['VHAQ-U', 'Viveon Health Acquisition Corp - Units (1 Ord 1 War & 1 Right )', 'NYSE MKT', 'Stock', '2020-12-23', 'null', 'Active']
['VHAQ-WS', 'Viveon Health Acquisition Corp - Warrants (01/01/9999)', 'NYSE MKT', 'Stock', '2021-02-04', 'null', 'Active']
['VHC', 'Virnetx Holding Corp', 'NYSE', 'Stock', '1999-07-30', 'null', 'Active']
['VHI', 'Valhi Inc', 'NYSE', 'Stock', '1986-06-04', 'null', 'Active']
['VHNAU', 'Vahanna Tech Edge Acquisition I Corp - Units (1 Ord Class A & 1/2 War)', 'NASDAQ', 'Stock', '2021-11-23', 'null', 'Active']
['VHT', 'Vanguard Health Care ETF', 'NYSE ARCA', 'ETF', '2004-01-30', 'null', 'Active']
['VIA', 'Via Renewables Inc', 'NASDAQ', 'Stock', '2014-07-29', 'null', 'Active']
['VIAC', 'ViacomCBS Inc - Class B', 'NASDAQ', 'Stock', '2006-01-03', 'null', 'Active']
['VIACA', 'ViacomCBS Inc - C

In [29]:
import pandas as pd
data=pd.DataFrame(my_list)
#df.set_index([0], inplace=True)
#df
data.columns=data.iloc[0].values
data=data.drop(0)
data=data.reset_index(drop=True)
data

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,null,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,null,Active
2,AAA,AAF First Priority CLO Bond ETF,NYSE ARCA,ETF,2020-09-09,null,Active
3,AAAU,Goldman Sachs Physical Gold ETF,NYSE ARCA,ETF,2018-08-15,null,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,null,Active
...,...,...,...,...,...,...,...
12023,ZYME,Zymeworks Inc,NYSE,Stock,2017-04-28,null,Active
12024,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,null,Active
12025,ZYXI,Zynex Inc,NASDAQ,Stock,2002-12-31,null,Active
12026,ZZK,,NYSE ARCA,Stock,2020-07-22,null,Active


Your API key is: RA9208IQ9J5IF8AS

Earnings Calendar

This API returns a list of company earnings expected in the next 3, 6, or 12 months.

In [37]:
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=RA9208IQ9J5IF8AS'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
   # for row in my_list:
        #print(row)

import pandas as pd
data=pd.DataFrame(my_list)
data.columns=data.iloc[0].values
data=data.drop(0)
data=data.reset_index(drop=True)
data

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,A,Agilent Technologies Inc,2022-02-14,2022-01-31,1.17,USD
1,AA,Alcoa Corp,2022-01-18,2021-12-31,2.06,USD
2,AADI,Aadi Bioscience Inc,2022-03-09,2021-12-31,-1.06,USD
3,AAIC,Arlington Asset Investment Corp - Class A,2022-02-14,2021-12-31,0.06,USD
4,AAL,American Airlines Group Inc,2022-01-26,2021-12-31,-1.76,USD
...,...,...,...,...,...,...
4250,ZUO,Zuora Inc - Class A,2022-03-09,2022-01-31,-0.02,USD
4251,ZVO,Zovio Inc,2022-02-22,2021-12-31,-0.11,USD
4252,ZYME,Zymeworks Inc,2022-02-22,2021-12-31,-1.19,USD
4253,ZYNE,Zynerba Pharmaceuticals Inc,2022-03-08,2021-12-31,-0.26,USD


This API returns the annual and quarterly income statements for the company of interest, with normalized fields mapped to GAAP and IFRS taxonomies of the SEC. Data is generally refreshed on the same day a company reports its latest earnings and financials.

In [19]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=IBM&apikey=RA9208IQ9J5IF8AS'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2020-12-31', 'reportedCurrency': 'USD', 'grossProfit': '35575000000', 'totalRevenue': '73620000000', 'costOfRevenue': '38046000000', 'costofGoodsAndServicesSold': '439000000', 'operatingIncome': '4609000000', 'sellingGeneralAndAdministrative': '23082000000', 'researchAndDevelopment': '6333000000', 'operatingExpenses': '30966000000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-1288000000', 'interestIncome': '105000000', 'interestExpense': '1288000000', 'nonInterestIncome': 'None', 'otherNonOperatingIncome': '-861000000', 'depreciation': '4227000000', 'depreciationAndAmortization': '2468000000', 'incomeBeforeTax': '4726000000', 'incomeTaxExpense': '-864000000', 'interestAndDebtExpense': '1288000000', 'netIncomeFromContinuingOperations': '5501000000', 'comprehensiveIncomeNetOfTax': '4850000000', 'ebit': '6014000000', 'ebitda': '8482000000', 'netIncome': '5590000000'}, {'fiscalDateEnding': '2019-12-31', 'reportedCurrency':

In [23]:
import requests
import json
import os
#url = "你的api请求"
#r = requests.get(url)
# 将api请求结果的json格式以字典形式保存
#result = r.json()
#print("Status code:",r.status_code)
#print("Total weather information:", result)
# 删除已有的result.txt
#dir = "/.result.txt"
#if os.path.exists(dir):
#    os.remove("dir")
with open("json.txt", "w") as fp:
    fp.write(json.dumps(data,indent=4, ensure_ascii=False))


In [32]:

#!/usr/bin/python3
# -*- coding: utf-8 -*-

import csv

# 获取ｊｓｏｎ数据
import json

with open('json.txt', 'r') as f:
    rows = json.load(f)

rows['annualReports']



[{'fiscalDateEnding': '2020-12-31',
  'reportedCurrency': 'USD',
  'grossProfit': '35575000000',
  'totalRevenue': '73620000000',
  'costOfRevenue': '38046000000',
  'costofGoodsAndServicesSold': '439000000',
  'operatingIncome': '4609000000',
  'sellingGeneralAndAdministrative': '23082000000',
  'researchAndDevelopment': '6333000000',
  'operatingExpenses': '30966000000',
  'investmentIncomeNet': 'None',
  'netInterestIncome': '-1288000000',
  'interestIncome': '105000000',
  'interestExpense': '1288000000',
  'nonInterestIncome': 'None',
  'otherNonOperatingIncome': '-861000000',
  'depreciation': '4227000000',
  'depreciationAndAmortization': '2468000000',
  'incomeBeforeTax': '4726000000',
  'incomeTaxExpense': '-864000000',
  'interestAndDebtExpense': '1288000000',
  'netIncomeFromContinuingOperations': '5501000000',
  'comprehensiveIncomeNetOfTax': '4850000000',
  'ebit': '6014000000',
  'ebitda': '8482000000',
  'netIncome': '5590000000'},
 {'fiscalDateEnding': '2019-12-31',
  '

In [34]:
# 创建文件对象
f = open('data.csv', 'w')

# 通过文件创建csv对象
csv_write = csv.writer(f)

# writerow: 按行写入，　writerows: 是批量写入
# 写入数据 取列表的第一行字典，用字典的ｋｅｙ值做为头行数据
csv_write.writerow(rows['annualReports'][0].keys())

# 循环里面的字典，将value作为数据写入进去
for row in rows['annualReports']:
    csv_write.writerow(row.values())
    
# 关闭打开的文件
f.close()


In [41]:
import json#导入json库
import openpyxl

def json_to_excel(json_file,excel_file):
    #读取json文件数据
    with open(json_file, mode='r', encoding='utf-8') as jf:
        json_data= json.load(jf)
    k = json_data.keys()
    wb = Workbook()#创建excel文件
    for sheet_name in k:
        try:
            wb.remove(sheet_name)  # 如表已存在则移除工作表
        except:
            pass
        wb.create_sheet(sheet_name, 0)  # 创建表
        ws = wb[sheet_name]  # 操作指定表
        sheet_data = json_data[sheet_name]  # 获取表要写入的数据
        for t in range(1, len(sheet_data) + 1):  # 遍历要写入的行数
            i = 65  # ASCII码'65'表示'A'
            for j in range(1, len(sheet_data[t - 1]) + 1):  # 遍历每行要写入的数量
                ws['%s%d' % (chr(i), t)] = sheet_data[t - 1][j - 1]
                i += 1
    wb.save(excel_file)

json_to_excel(r'./test,json',r'./test.xlsx')#调用函数，传入参数

FileNotFoundError: [Errno 2] No such file or directory: './test,json'

In [ ]:

#!/usr/bin/python3
# -*- coding: utf-8 -*-

import csv

# 获取ｊｓｏｎ数据
import json

# 创建文件对象
f = open('data.csv', 'w')

# 通过文件创建csv对象
csv_write = csv.writer(f)

# writerow: 按行写入，　writerows: 是批量写入
# 写入数据 取列表的第一行字典，用字典的ｋｅｙ值做为头行数据
csv_write.writerow(rows[0].keys())

# 循环里面的字典，将value作为数据写入进去
for row in rows:
    csv_write.writerow(row.values())
    
# 关闭打开的文件
f.close()



In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [16]:
response = requests.get("https://cd.58.com/ershoufang/?PGTID=0d200001-0006-6d1b-5ed5-d23a7edbbe42&ClickID=1")
#print (response.text)

In [17]:
html = response.text
html_tree = bs(html, "html.parser")
print (html_tree)

<!DOCTYPE doctype html>

<html lang="en">
<head>
<title>成都二手房网，成都房产网，成都二手房买卖出售交易信息-成都58同城</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1, maximum-scale=1.0, minimum-scale=1.0,user-scalable=no,viewport-fit=cover" data-n-head="ssr" name="viewport"/><meta content="province=四川;city=成都;coord=104.072138,30.578829" data-n-head="ssr" name="location"/><meta content="format=html5; url=https://m.58.com/cd/ershoufang/" data-n-head="ssr" name="mobile-agent"/><meta content="pc" data-n-head="ssr" name="applicable-device"/><meta content="成都二手房，成都房产网，成都二手房出售，成都二手房买卖信息" data-n-head="ssr" name="keywords"/><meta content="58同城成都二手房网为您提供成都二手房出售，成都个人二手房、经纪人发布成都二手房买卖交易信息，包括成都商品房价格，成都次新房，想在成都买房购房、成都卖房就来58同城房产网。" data-n-head="ssr" name="description"/><link data-n-head="ssr" href="//pages.anjukestatic.com/usersite/touch/img/favicon-wb.ico" rel="icon" type="image/x-icon"/><link data-n-head="ssr" href="https://m.58.com/cd/ershoufang/" media="only screen and(max-w

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(host_list)? (<ipython-input-6-7abe87f4338b>, line 6)

In [8]:
#from autoGetRegisDatas.writeCsv import write_csv
from bs4 import BeautifulSoup
from urllib import request
import os
import re


def get_soup(url):
    """
    将页面用BeautifulSoup库处理
    :return:
    """
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:59.0) Gecko/20100101 Firefox/59.0"
    }
    req = request.Request(url, headers=header)
    html = request.urlopen(req).read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    return soup


def get_page_url(url):
    """
    从登记结束页面爬取所有分页url，存入列表中
    :param url:
    :return:
    """
    page_url = []
    paging = get_soup(url).find_all(href=re.compile('page'))
    if paging is not []:
        for paging_i in range(len(paging)-1):
            page_url.append(url.split('xmgs')[0] + paging[paging_i]['href'])

    return page_url


def get_result_url(url):
    """
    从登记结束页面爬取所有的结果公示url
    :return:
    """
    result_url = []
    result_list = get_soup(url).find_all('span', string='结果公示')
    for result_list_i in result_list:
        result_url.append(url.split('xmgs')[0] + result_list_i['onclick'].split("='")[1].split("'")[0])

    if get_page_url(url) is not []:
        for page_i in get_page_url(url):
            result_list = get_soup(page_i).find_all('span', string='结果公示')
            for result_list_i in result_list:
                result_url.append(url.split('xmgs')[0] + result_list_i['onclick'].split("='")[1].split("'")[0])

    return result_url


def get_regis_data(result_url):
    """
    从结果公示页面获取每个楼盘登记的数据
    :return:
    """
    for result_url_i in result_url:
        regis_data = []
        gs = get_soup(result_url_i)
        house_name = gs.find(string=re.compile('意向登记结果公示')).split(' 意向')[0].strip()
        regis_data.append(house_name)
        all_data = gs.find_all('font')
        for data_i in all_data:
            regis_data.append(data_i.string.strip())
        write_csv(regis_data)


os.remove('regisData.csv')
url1 = 'http://124.115.228.93/zfrgdjpt/xmgs.aspx?state=4'
url2 = 'http://124.115.228.93/zfrgdjpt/xmgsca.aspx?state=4'
lst = [url1, url2]
write_csv(['项目名', '房源数', '登记数', '资料核验数', '核验通过数', '刚需数', '普通数', '未通过数'])
for lst_i in lst:
    get_regis_data(get_result_url(lst_i))
os.system("openCsv.bat")
print('ok!')

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: 'regisData.csv'

In [18]:
import urllib.request
from bs4 import BeautifulSoup
import re
def getHtml(url):
    html=urllib.request.urlopen(url).read()
    html= html.decode('utf-8')
    return html
html = getHtml("http://news.sina.com.cn")
bsObj = BeautifulSoup(html, "html.parser")#指定Beautiful的解析器为“html.parser”
downloadList=bsObj.select('a')
#标签名不加任何修饰，类名前加点，id名前加 #，在这里我们也可以利用类似的方法来筛选元素，用到的方法是 soup.select()，返回类型是 list
text_re=re.compile(r'(?<=_blank">)\S+(?=</a>)')
text_list=[]
for txt in downloadList:
    html="{}".format(txt)
    text_list+=text_re.findall(html)
print(text_list)


['<b>新闻</b>', '军事', '国内', '国际', '<b>财经</b>', '股票', '基金', '外汇', '<b>科技</b>', '手机', '探索', '众测', '<b>体育</b>', 'NBA', '英超', '中超', '<b>娱乐</b>', '明星', '电影', '星座', '<b>汽车</b>', '报价', '买车', '新车', '<b>博客</b>', '专栏', '历史', '天气', '<b>视频</b>', '综艺', 'VR', '直播', '<b>房产</b>', '二手房', '家居', '收藏', '<b>时尚</b>', '女性', '医药', '育儿', '<b>教育</b>', '留学', '公益', '佛学', '<b>图片</b>', '读书', '黑猫', '司法', '<b>微博</b>', '城市', '上海', '学投资', '<b>旅游</b>', '文化', '彩票', '高尔夫', '<b>游戏</b>', '手游', '邮箱', 'English', '交易', '理财', '政务', '车险', '页游', 'SHOW', '搜索', '客服', '导航', '新浪新闻客户端', '新浪财经客户端', '新浪体育客户端', '新浪众测客户端', '新浪公开课', '新浪邮箱客户端', '手机新浪网', '新闻', '军事', '西安开展全市范围环境消杀', '精彩纷呈：军事高清图集汇总', '新浪放映厅', '新浪资讯台', '军事视频', '火炮炮口为何会有各种怪“帽子”？视频告诉你', '嘘！你听', '新闻图片', '更多', '坚持党的领导', '中国航天员将首次太空跨年！', '明年房地产怎么干？住建部部长说了一个字', '西安市新增155例确诊病例活动轨迹公布', '山东平度一官员威胁上访者？停职并道歉！', '广西东兴市公布7例确诊病例轨迹：涉KTV、酒店等', '内蒙古满洲里125个小区等调整为防范区', '飙涨800%！普京：只能怪你们自己', '沙特主导的多国联军再次空袭也门首都萨那', '全长1687米！天巴高速首座长隧道顺利贯通', '以色列将投资逾3亿美元加强戈兰高地犹太人社区建设', '2021年终观察：读懂中国外交五个“新”', '利比亚最高国家委员